In [5]:
%load_ext autoreload
%autoreload 2

!pip3 install plotly

import plotly.graph_objects as go
import importlib
import sys
from urllib.request import urlretrieve


# python libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from sympy import Matrix

import pydrake.all

# pydrake imports
from pydrake.common.containers import namedview
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.all import (Variable, SymbolicVectorSystem, VectorSystem, DiagramBuilder,
                         LogOutput, Simulator, ConstantVectorSource,
                         MathematicalProgram, Solve, SnoptSolver, PiecewisePolynomial, eq, cos, sin, 
                         DirectTranscription, DirectCollocation)
from pydrake.all import (FiniteHorizonLinearQuadraticRegulator,  MakeFiniteHorizonLinearQuadraticRegulator)

from pydrake.systems.framework import (BasicVector, BasicVector_, LeafSystem_,
                                       LeafSystem)
import pydrake.symbolic as sym

from lyapunov_control import *
from trajopt_control import *
from robust_trajopt import *

# Matplotlib parameters
plt.rcParams['font.size'] = '16'
plt.rcParams['lines.linewidth'] = 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
x0 = np.array([10, 0, np.pi-0.2])
xf = np.array([0, 0, 0])

r = 2.
simulator, traj, control = lyapunov_simulation(x0, xf)

In [34]:
def animate_traj(traj, title_text):
    N = np.shape(traj)[1]
    # Generate curve data
    t = np.linspace(0, N)
    x = traj[0,:]
    y = traj[1,:]
    xm = np.min(x) - 1.5
    xM = np.max(x) + 1.5
    ym = np.min(y) - 1.5
    yM = np.max(y) + 1.5
    # s = np.linspace(-1, 1, N)
    # xx = s + s ** 2
    # yy = s - s ** 2


    # Create figure
    fig = go.Figure(
        data=[go.Scatter(x=x, y=y,
                         mode="lines",
                         line=dict(width=2, color="blue")),
              go.Scatter(x=x, y=y,
                         mode="lines",
                         line=dict(width=2, color="blue"))],
        layout=go.Layout(
            xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
            yaxis=dict(range=[ym, yM], autorange=False, zeroline=False, scaleanchor="x", scaleratio=1),
            title_text=title_text, hovermode="closest",
            updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                            method="animate",
                                            args=[None, 
                                                  {"frame": {"duration": 1}}, 
                                                 {"transition": {"duration": 1}}])])]),
        frames=[go.Frame(
            data=[go.Scatter(
                x=[x[k]],
                y=[y[k]],
                mode="markers",
                marker=dict(color="red", size=10))])

            for k in range(N)]
    )

    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1
    fig.show()
    return fig

fig = animate_traj(traj, "Lyapunov Controller")
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.write_html("figures/lyapunov_controller.html", animation_opts={
    "frame": {
        "duration": 1 
    }

})


In [35]:
x_opt, u_opt = trajopt_simulation(x0, xf, 0.5) # simulate from initial point (x0) to final point (xf)

fig2 = animate_traj(x_opt, "Optimal Controller")
fig2.write_html("figures/optimal_controller.html", animation_opts={
    "frame": {
        "duration": 1 
    }

})


In [39]:
x0 = np.array([10, 0.5, np.pi-0.3])
x_traj, u_traj = robust_trajopt(x_opt, u_opt, x0)

fig3 = animate_traj(x_traj, "Robust Controller")
fig3.write_html("figures/robust_controller.html", animation_opts={
    "frame": {
        "duration": 1 
    }

})
